In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
from csv import reader
from sklearn.model_selection import train_test_split

# Load the Albanian summarization dataset
#dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation': 'validation.csv'})

allNews = []
with open('/content/final.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        allNews.append(row)

allNews.pop(0)
articleDataset = []
summaryDataset = []
for news in allNews:
  articleDataset.append(news[4])
  summaryDataset.append(news[2] + news[3])

# Split the data into training and test sets
train_article, test_article = train_test_split(articleDataset, test_size=0.2, random_state=42)
train_summary, test_summary = train_test_split(summaryDataset, test_size=0.2, random_state=42)


In [ ]:
import torch
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, MT5Config, AutoTokenizer
import csv
from csv import reader

# Load the pre-trained T5 tokenizer

# Load the pre-trained T5 model with the configuration for multi-lingual tasks
tokenizer = AutoTokenizer.from_pretrained('google/mt5-base', model_max_length=256)
model_config = MT5Config.from_pretrained('google/mt5-base', is_encoder_decoder=True, pad_token_id=tokenizer.pad_token_id)
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-base', config=model_config)


# Tokenize the input text and summary
input_ids = tokenizer.batch_encode_plus(train_article, padding=True, truncation=True, max_length=512, return_tensors="pt")
target_ids = tokenizer.batch_encode_plus(train_summary, padding=True, truncation=True, max_length=128, return_tensors="pt")

# Create a dataloader for the training data
train_dataset = torch.utils.data.TensorDataset(input_ids["input_ids"], input_ids["attention_mask"], target_ids["input_ids"], target_ids["attention_mask"])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4)

# Set the device to run the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Fine-tune the model
model.to(device)
for epoch in range(20):
    epoch_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, input_mask, target_ids, target_mask = [x.to(device) for x in batch]
        outputs = model(input_ids=input_ids, attention_mask=input_mask, labels=target_ids, decoder_attention_mask=target_mask)
        loss = criterion(outputs.logits.view(-1, outputs.logits.shape[-1]), target_ids.view(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print("Epoch %d Loss: %f" % (epoch, epoch_loss / len(train_loader)))
    
# Save the fine-tuned model
model.save_pretrained("t5-multilingual-summarization")
tokenizer.save_pretrained("t5-multilingual-summarization")


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Epoch 0 Loss: 2.169909
Epoch 1 Loss: 1.692467
Epoch 2 Loss: 1.395333
Epoch 3 Loss: 1.120637
Epoch 4 Loss: 0.895257
Epoch 5 Loss: 0.689601
Epoch 6 Loss: 0.507720
Epoch 7 Loss: 0.366829
Epoch 8 Loss: 0.394570
Epoch 9 Loss: 0.233784
Epoch 10 Loss: 0.134859
Epoch 11 Loss: 0.130828
Epoch 12 Loss: 0.136619
Epoch 13 Loss: 0.132968
Epoch 14 Loss: 0.120541
Epoch 15 Loss: 0.112305
Epoch 16 Loss: 0.103017
Epoch 17 Loss: 0.095034
Epoch 18 Loss: 0.087471
Epoch 19 Loss: 0.080990


('t5-multilingual-summarization/tokenizer_config.json',
 't5-multilingual-summarization/special_tokens_map.json',
 't5-multilingual-summarization/spiece.model',
 't5-multilingual-summarization/added_tokens.json',
 't5-multilingual-summarization/tokenizer.json')

In [ ]:

# rouge and bleu evaluation

import csv
from csv import reader
from transformers import T5Config, T5ForConditionalGeneration
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
import numpy as np


# Load the fine-tuned BART model and tokenizer
#model_path = "path/to/fine-tuned/model"
#tokenizer = BartTokenizer.from_pretrained(model_path)
#model = BartForConditionalGeneration.from_pretrained(model_path)98525256588
#  MODEL AND TOKENIZER IS GENERATED BEFORE

# Load the validation data
#validation_data = load_data("path/to/validation/data")
'''validation_data = []
allNews = []
with open('/content/final.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        allNews.append(row)

allNews.pop(0)

for news in allNews:
  validation_data.append([news[4], news[3]])'''


tokenizer = AutoTokenizer.from_pretrained('/content/t5-multilingual-summarization', model_max_length=256)
model_config = MT5Config.from_pretrained('/content/t5-multilingual-summarization', is_encoder_decoder=True, pad_token_id=tokenizer.pad_token_id)
model = MT5ForConditionalGeneration.from_pretrained('/content/t5-multilingual-summarization', config=model_config)

# Generate summaries for the validation data using the fine-tuned BART model
predicted_summaries = []
target_summaries = []
for article, summary in zip(test_article[:100], test_summary[:100]):
    #input_ids = tokenizer.encode([example[0]], return_tensors="pt")
    #summary_ids = model.generate(input_ids)
    inputs = tokenizer([article], padding=True, truncation=True, return_tensors='pt')
    summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=200)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    predicted_summaries.append(predicted_summary)
    target_summaries.append(summary)

'''tokenizer = AutoTokenizer.from_pretrained('/content/t5-multilingual-summarization', model_max_length=256)
model_config = MT5Config.from_pretrained('/content/t5-multilingual-summarization', is_encoder_decoder=True, pad_token_id=tokenizer.pad_token_id)
model = MT5ForConditionalGeneration.from_pretrained('/content/t5-multilingual-summarization', config=model_config)
text = 'Në një konferencë për mediat, Balluku pohoi se ka pasur diskutime lidhur me këtë çështje, por si vendim i takon Ministrisë së Financave.Ajo shtoi se nga ana e qeverisë, një vendim i tillë është i mirëpritur, duke qenë se çmimi i karburanteve në bursë është stabilizuar."Bordi nuk ka synim të lehtësoj kostot, por transparencën. Bordi nuk vendos dhe as nuk e ul dhe as nuk e ngre çmimin. Por bën transparente metodologjinë. Se ka një bursë dhe përmes kësaj metodologjie vendoset çmimi tavan. Bursën nuk e lëviz as topi. Bordi mblidhet kur ka ndryshim çmimi.Nuk e vendosim dot ne se kur mblidhet bordi. Duke qenë që çmimi është stabilizuar, kemi një diskutim për ndalimin e të funksionuarit. I përket Ministrisë së Financave dhe do jetë i mirëpritur si vendim. Shteti do të monitoroj edhe nëse bordi nuk do të jetë ekzistues" , deklaroi Balluku.'
target_summary = 'Ministrja e Infrastrukturës dhe Energjitikës, Belinda Balluku është pyetur sot nëse po diskutohet mbyllja e Bordit të Transparencës së Çmimeve të Karburanteve.'
inputs = tokenizer([text], padding=True, truncation=True, return_tensors='pt')
summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=120)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
target_summaries.append(target_summary)
predicted_summaries.append(summary)'''

#print(summary)

# Compute the ROUGE metrics
rouge = Rouge()
rouge_scores = rouge.get_scores(predicted_summaries, target_summaries, avg=True)

print("ROUGE scores:")
print(rouge_scores)
score_1 = round(rouge_scores['rouge-1']['f'], 2)    
score_2 = round(rouge_scores['rouge-2']['f'], 2)    
score_L = round(rouge_scores['rouge-l']['f'], 2)    
print("rouge1:", score_1, "| rouge2:", score_2, "| rougeL:",
         score_2, "--> avg rouge:", round(np.mean(
         [score_1,score_2,score_L]), 2))

# Compute the BLEU metrics
bleu_scores = corpus_bleu([[summary] for summary in target_summaries], predicted_summaries)

print("BLEU score:")
print(bleu_scores)

ROUGE scores:
{'rouge-1': {'r': 0.22274899740516532, 'p': 0.25248400968364965, 'f': 0.23118924519976797}, 'rouge-2': {'r': 0.04472634279074775, 'p': 0.04880295577003707, 'f': 0.04552741852885627}, 'rouge-l': {'r': 0.18368945699010603, 'p': 0.2074810152668575, 'f': 0.19032311482281425}}
rouge1: 0.23 | rouge2: 0.05 | rougeL: 0.05 --> avg rouge: 0.16
BLEU score:
0.3634692485546286


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/t5-multilingual-summarization', model_max_length=256)
model_config = MT5Config.from_pretrained('/content/t5-multilingual-summarization', is_encoder_decoder=True, pad_token_id=tokenizer.pad_token_id)
model = MT5ForConditionalGeneration.from_pretrained('/content/t5-multilingual-summarization', config=model_config)
predicted_summaries = []
target_summaries = []

#text = 'Në një konferencë për mediat, Balluku pohoi se ka pasur diskutime lidhur me këtë çështje, por si vendim i takon Ministrisë së Financave.Ajo shtoi se nga ana e qeverisë, një vendim i tillë është i mirëpritur, duke qenë se çmimi i karburanteve në bursë është stabilizuar."Bordi nuk ka synim të lehtësoj kostot, por transparencën. Bordi nuk vendos dhe as nuk e ul dhe as nuk e ngre çmimin. Por bën transparente metodologjinë. Se ka një bursë dhe përmes kësaj metodologjie vendoset çmimi tavan. Bursën nuk e lëviz as topi. Bordi mblidhet kur ka ndryshim çmimi.Nuk e vendosim dot ne se kur mblidhet bordi. Duke qenë që çmimi është stabilizuar, kemi një diskutim për ndalimin e të funksionuarit. I përket Ministrisë së Financave dhe do jetë i mirëpritur si vendim. Shteti do të monitoroj edhe nëse bordi nuk do të jetë ekzistues" , deklaroi Balluku.'
#target_summary = 'Ministrja e Infrastrukturës dhe Energjitikës, Belinda Balluku është pyetur sot nëse po diskutohet mbyllja e Bordit të Transparencës së Çmimeve të Karburanteve.'

text = "Rama tha se ai është turpi më i madh që vendi ynë ka pas në krye, ndërsa shtoi se kush e voton atë turpëron veten, fëmijët, familjen, prindërit, të gjallë e të vdekur në shtëpinë e tij.Rama: Në 25 prill çoni në shtëpi Ilir Metën. Boll, mjaftë! Ilir Meta duhet të shkojë në shtëpi. Ilir Meta është turpi më i madh që Shqipëria ka mundur të ketë në krye, ai turp duhet zhbërë në 25 prill dhe turp kush voton Ilir Metën, turpëron veten, fëmijët, familjen, prindërit, të gjallë e të vdekur në shtëpinë e tij. Boll se nuk është Shqipëria peng i një njeriu që ka ikur dhe i ka lëshuar pëllumba dhe sorra bashkë."
target_summary = "Në 25 prill çojeni në shtëpi - Rama ashpërson tonet: Ilir Meta turpi më i madh që Shqipëria ka pas në krye! Kush e voton turpëron të gjallë e të... Kryeministri Edi Rama pak para mbylljes së fushatës elektorale në Tiranë ka ashpërsuar tonet ndaj presidentit të Republikës, Ilir Meta."
inputs = tokenizer([text], padding=True, truncation=True, return_tensors='pt')
summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=120)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
target_summaries.append(target_summary)
predicted_summaries.append(summary)

print(summary)

“Ilir Meta, peng i një njeriu që ka lëshuar pëllumba dhe sorra bashkë”- Rama për Ilir Metën: Turpunoni veten, prindërit dhe fëmijëtKryetari i Partisë së Lirisë, Ilir Meta u shpreh në një intervistë për “News 24” se Partia Demokratike është peng i një njeriu që ka lëshuar në shtëpinë e tij një stemë për Ilir Metën.
